# Eve hands-on day 2

You start from a high level IR representing the following DSL

```python
def fun(out_field, in_field):
    out_field = -4 * in_field[0,0] + in_field[-1,0] + in_field[1,0] + in_field[0,-1] + in_field[0,1]
```

- Write a pass: extent analysis
- Lower from HIR to LIR

The HIR is provided in the file `hir.py` as well as the implementation of the Laplacian in that dialect (see below).
Feel free to change the skeleton for `ExtentAnalysis` and `HIRtoLIR` as you like.

## Preparation

Create a new file called `lir.py` and put your dialect from *Day 1* into that file. For simplicity also put the code generator in this file.

In [2]:
# Definition of the Laplacian in HIR (don't change)

import hir
from hir import *

lap_expr = BinaryOp(left=BinaryOp(left=Literal(value="-4"), right=FieldAccess(name="in", offset=Offset.zero()), op="*"), right=BinaryOp(left=BinaryOp(left=FieldAccess(name="in", offset=Offset(i=-1,j=0)),right=FieldAccess(name="in", offset=Offset(i=1,j=0)),op="+"),right=BinaryOp(left=FieldAccess(name="in", offset=Offset(i=0,j=-1)),right=FieldAccess(name="in", offset=Offset(i=0,j=1)),op="+"),op="+"), op="+")
assign = AssignStmt(left=FieldAccess(name="out", offset=Offset.zero()), right=lap_expr)
stencil = Stencil(name="lap", params=[FieldParam(name="out"), FieldParam(name="in")], body=[assign])

from devtools import debug
debug(stencil)

<ipython-input-2-6c194486fea7>:11 <module>
    stencil: Stencil(
        id_='Stencil_22',
        name='lap',
        params=[
            FieldParam(
                id_='FieldParam_20',
                name='out',
            ),
            FieldParam(
                id_='FieldParam_21',
                name='in',
            ),
        ],
        body=[
            AssignStmt(
                id_='AssignStmt_19',
                left=FieldAccess(
                    id_='FieldAccess_18',
                    name='out',
                    offset=Offset(
                        id_='Offset_17',
                        i=0,
                        j=0,
                    ),
                ),
                right=BinaryOp(
                    id_='BinaryOp_16',
                    left=BinaryOp(
                        id_='BinaryOp_4',
                        left=Literal(
                            id_='Literal_1',
                            value='-4',
                       

In [4]:
# Feel free to change the signature of the provided methods (or remove them) if you prefer a different structure.
import eve

class Extent(eve.Model):
    i_left: int
    i_right: int
    j_left: int
    j_right: int

    @classmethod
    def zero(cls):
        return cls(i_left = 0, i_right = 0, j_left = 0, j_right = 0)

    def __add__(self, other):
        return type(self)(i_left=min(self.i_left, other.i_left), i_right=max(self.i_right, other.i_right),j_left=min(self.j_left, other.j_left), j_right=max(self.j_right, other.j_right))

    @classmethod
    def from_offset(cls, offset: Offset):
        return cls(i_left=min(0, offset.i), i_right=max(0,offset.i), j_left=min(0,offset.j), j_right=max(0, offset.j))


class ExtentAnalysis(eve.NodeVisitor):
    @classmethod
    def apply(cls, root: Stencil, **kwargs):
        return cls().visit(root)

In [5]:
# Feel free to change the signature of the provided methods (or remove them) if you prefer a different structure.

import eve
import lir

class HIRToLIR(eve.NodeTranslator):
    def __init__(self, extents, **kwargs):
        self.extents = extents

    @classmethod
    def apply(cls, root: Stencil, extents, **kwargs):
        hir_to_lir = cls(extents)
        return hir_to_lir.visit(root)

    def visit_Literal(self, node: Literal, **kwargs):
        return lir.Literal(value=node.value)

    def visit_BinaryOp(self, node: BinaryOp, **kwargs):
        return lir.BinaryOp(left=self.visit(node.left), right=self.visit(node.right), op=node.op)
    
    # TODO 

In [7]:
# No need to change anything here (except if you changed names of passes or signature of apply)

extents = ExtentAnalysis.apply(stencil)
lir_stencil = HIRToLIR.apply(stencil, extents)
debug(lir_stencil)

AttributeError: 'HIRToLIR' object has no attribute 'memo'

In [8]:
# Don't touch
import os

cppcode = lir.LIR_to_cpp.apply(lir_stencil)
formatted_code = eve.codegen.format_source("cpp", cppcode, style="LLVM")
print(formatted_code)

output_file = "generated.hpp"
with open(output_file, "w+") as output:
    output.write(formatted_code)

NameError: name 'lir_stencil' is not defined

In [9]:
!g++ driver.cc -o lap
!./lap

In file included from driver.cc:5:
generated.hpp:1:2: error: expected unqualified-id before numeric constant
    1 | (1+1)
      |  ^
generated.hpp:1:2: error: expected ‘)’ before numeric constant
    1 | (1+1)
      | ~^
      |  )
/bin/bash: ./lap: No such file or directory
